In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
import pandas as pd
import shap
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris, load_breast_cancer, fetch_california_housing
from alibi.explainers import Counterfactual, CounterfactualProto

In [2]:
tf.compat.v1.reset_default_graph()

cali = fetch_california_housing(as_frame=True)
X = cali['data'].values
target = cali['target'].values
y = np.zeros((target.shape[0]))
y[np.where(target > np.median(target))[0]] = 1
feature_names = cali['feature_names']
X.shape, y.shape

((20640, 8), (20640,))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((16512, 8), (4128, 8), (16512,), (4128,))

In [4]:
model = XGBClassifier(n_estimators=1000, learning_rate=0.1, max_depth=3, reg_alpha=1)
model.fit(X_train, y_train)
model.score(X_train, y_train), model.score(X_test, y_test)

(0.9464631782945736, 0.903343023255814)

In [5]:
cf = CounterfactualProto(model.predict_proba, shape=(1, X_test.shape[1]), use_kdtree=True, feature_range=(X_train.min(axis=0), X_train.max(axis=0)),
                         c_init=1, c_steps=10, theta=1)
cf.fit(X_train)

No encoder specified. Using k-d trees to represent class prototypes.


CounterfactualProto(meta={
  'name': 'CounterfactualProto',
  'type': ['blackbox', 'tensorflow', 'keras'],
  'explanations': ['local'],
  'params': {
              'kappa': 0.0,
              'beta': 0.1,
              'gamma': 0.0,
              'theta': 1,
              'cat_vars': None,
              'ohe': False,
              'use_kdtree': True,
              'learning_rate_init': 0.01,
              'max_iterations': 1000,
              'c_init': 1,
              'c_steps': 10,
              'eps': (0.001, 0.001),
              'clip': (-1000.0, 1000.0),
              'update_num_grad': 1,
              'write_dir': None,
              'feature_range': (array([   0.4999    ,    1.        ,    0.88888889,    0.33333333,
          3.        ,    0.69230769,   32.54      , -124.35      ]), array([ 1.50001000e+01,  5.20000000e+01,  1.41909091e+02,  3.40666667e+01,
        3.56820000e+04,  1.24333333e+03,  4.19500000e+01, -1.14310000e+02])),
              'shape': (1, 8),
            

In [6]:
instance_to_explain = X_test[[0]]
original_prediction_idx = model.predict(instance_to_explain)[0]
target_prediction_as_array = 1 - model.predict(X_test[[0]])
instance_to_explain, original_prediction_idx, target_prediction_as_array

(array([[   1.475     ,   45.        ,    3.30656934,    1.06569343,
          749.        ,    5.46715328,   37.88      , -122.3       ]]),
 0,
 array([1]))

In [7]:
explanation = cf.explain(instance_to_explain, target_class=target_prediction_as_array)
explanation

No counterfactual found!


Explanation(meta={
  'name': 'CounterfactualProto',
  'type': ['blackbox', 'tensorflow', 'keras'],
  'explanations': ['local'],
  'params': {
              'kappa': 0.0,
              'beta': 0.1,
              'gamma': 0.0,
              'theta': 1,
              'cat_vars': None,
              'ohe': False,
              'use_kdtree': True,
              'learning_rate_init': 0.01,
              'max_iterations': 1000,
              'c_init': 1,
              'c_steps': 10,
              'eps': (0.001, 0.001),
              'clip': (-1000.0, 1000.0),
              'update_num_grad': 1,
              'write_dir': None,
              'feature_range': (array([   0.4999    ,    1.        ,    0.88888889,    0.33333333,
          3.        ,    0.69230769,   32.54      , -124.35      ]), array([ 1.50001000e+01,  5.20000000e+01,  1.41909091e+02,  3.40666667e+01,
        3.56820000e+04,  1.24333333e+03,  4.19500000e+01, -1.14310000e+02])),
              'shape': (1, 8),
              'is_mo

In [8]:
explanation.data.keys()

dict_keys(['cf', 'all', 'orig_class', 'orig_proba', 'id_proto'])

In [9]:
print(explanation.cf)

None


In [10]:
for i in range(0, 10):
    instance_to_explain = X_test[[i]]
    original_prediction_idx = model.predict(instance_to_explain)[0]
    target_prediction_as_array = 1 - model.predict(X_test[[0]])
    explanation = cf.explain(instance_to_explain, target_class=target_prediction_as_array)
    if explanation.cf is None:
        print(f'X_Test{i}: No counterfactual found!')
    else:
        print(f'X_Test{i}: {(explanation.cf["X"] - instance_to_explain).round(2)}')

No counterfactual found!


X_Test0: No counterfactual found!


No counterfactual found!


X_Test1: No counterfactual found!


No counterfactual found!


...

X_Test7: No counterfactual found!


No counterfactual found!


X_Test8: No counterfactual found!


No counterfactual found!


X_Test9: No counterfactual found!


In [11]:
tf.compat.v1.reset_default_graph()

# --- 1. 데이터 로드 및 분류 문제로 변환 ---
housing = fetch_california_housing(as_frame=True)
X = housing.data
y_regr = housing.target # 원래의 연속형 타겟 (집값)

# 타겟 변수를 중앙값 기준으로 0과 1로 변환
median_price = y_regr.median()
y_clf = (y_regr > median_price).astype(int)
target_names = ['Low Price', 'High Price']

print(f"집값 중앙값: {median_price:.2f}")
print(f"클래스 분포:\n{y_clf.value_counts()}")
print("="*50)

# 훈련/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y_clf, test_size=0.2, random_state=42)

# --- 2. XGBoost 분류 모델 학습 ---
model = XGBClassifier(n_estimators=1000, learning_rate=0.1, max_depth=3, reg_alpha=1)
model.fit(X_train, y_train)


# --- 3. 설명할 인스턴스 선택 ---
# 예시: 모델이 'High Price'(Class 1)로 예측한 데이터 중 하나를 찾습니다.
high_price_indices = np.where(model.predict(X_test) == 1)[0]
if len(high_price_indices) == 0:
    raise ValueError("테스트 데이터에서 'High Price'로 예측된 샘플이 없습니다.")
    
instance_index = high_price_indices[0]
instance_to_explain = X_test.iloc[[instance_index]]
original_prediction_idx = model.predict(instance_to_explain)[0]

print(f"설명 대상 샘플 인덱스: {instance_index}")
print(f"원본 데이터의 예측: Class {original_prediction_idx} ({target_names[original_prediction_idx]})")
print("="*50)


# --- 4. Alibi CounterFactualProto 설명기 생성 ---
# 목표: 예측을 'Low Price'(Class 0)으로 바꾸고 싶다
target_class = 0

# 각 피처가 가질 수 있는 현실적인 범위를 (min, max) 형태로 지정
feature_range = (X_train.values.min(axis=0), X_train.values.max(axis=0))

# CounterFactualProto 클래스 생성
cf_explainer = CounterfactualProto(
    model.predict_proba,
    shape=(1, X_train.shape[1]),
    feature_range=feature_range,
)

# 데이터 분포 및 프로토타입 학습
cf_explainer.fit(X_train.values)


# --- 5. 카운터팩추얼 설명 생성 ---
print("카운터팩추얼을 탐색합니다...")
# explain 메서드에 target_class 파라미터 사용
explanation = cf_explainer.explain(
    instance_to_explain.values,
    target_class=target_class
)


# --- 6. 결과 확인 ---
if explanation.cf is None:
    print("\n지정된 조건 내에서 카운터팩추얼을 찾지 못했습니다.")
    print("-> 생성자의 max_iter 값을 높여서 다시 시도해보세요.")
else:
    counterfactual = explanation.cf['X']
    cf_prediction_idx = model.predict(counterfactual)[0]

    print(f"\n카운터팩추얼의 예측: Class {cf_prediction_idx} ({target_names[cf_prediction_idx]})")
    print("\n--- 예측을 바꾸기 위해 변경된 피처 값 ---")

    diff = counterfactual - instance_to_explain.values
    changed_indices = np.where(np.abs(diff[0]) > 1e-4)[0]

    if len(changed_indices) == 0:
        print("최소한의 변경으로 예측을 바꿀 수 없어, 원본과 동일한 값이 반환되었습니다.")
    else:
        for idx in changed_indices:
            print(f"- {feature_names[idx]}: {instance_to_explain.values[0, idx]:.2f}  ->  {counterfactual[0, idx]:.2f}")

집값 중앙값: 1.80
클래스 분포:
MedHouseVal
0    10323
1    10317
Name: count, dtype: int64
설명 대상 샘플 인덱스: 2
원본 데이터의 예측: Class 1 (High Price)
카운터팩추얼을 탐색합니다...


No counterfactual found!



지정된 조건 내에서 카운터팩추얼을 찾지 못했습니다.
-> 생성자의 max_iter 값을 높여서 다시 시도해보세요.


In [ ]:
tf.compat.v1.reset_default_graph()

from sksurv.ensemble import RandomSurvivalForest
from sksurv.datasets import load_whas500

# --- 1. 데이터 로드 및 생존 분석 모델 학습 ---
X, y = load_whas500()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sksurv_model = RandomSurvivalForest(n_estimators=100, random_state=42)
sksurv_model.fit(X_train, y_train)

# --- 2. 설명할 인스턴스 선택 ---
# 예시: 위험 점수가 높은 직원(환자) 선택
instance_to_explain = X_test.iloc[[0]].values
original_risk_score = sksurv_model.predict(instance_to_explain)[0]
print(f"원본 데이터의 예측 위험 점수: {original_risk_score:.2f}")
print("="*50)

# --- 3. Alibi 카운터팩추얼 설명기 생성 ---
# Counterfactual 클래스는 회귀/분류 모두 동일하게 사용
cf_explainer = Counterfactual(
    sksurv_model.predict, # .predict는 위험 점수(숫자)를 반환하므로 회귀 문제와 동일
    shape=(1, X_train.shape[1]),
    max_iter=1000
)
cf_explainer.fit(X_train.values, y_train)

# --- 4. 카운터팩추얼 설명 생성 (목표 범위 지정) ---
print("카운터팩추얼을 탐색합니다...")

# 목표: 위험 점수를 0.5 이하의 안정적인 범위로 낮추기
# np.NINF는 음의 무한대를 의미
target_range = [np.NINF, 0.5] 

# target_class 대신 target_prediction 파라미터 사용
explanation = cf_explainer.explain(
    instance_to_explain,
    target_prediction=target_range
)

# --- 5. 결과 확인 ---
if explanation.cf is None:
    print(f"\n위험 점수를 {target_range[1]} 이하로 낮추는 대안을 찾지 못했습니다.")
else:
    counterfactual = explanation.cf['X']
    cf_risk_score = sksurv_model.predict(counterfactual)[0]

    print(f"\n카운터팩추얼의 예측 위험 점수: {cf_risk_score:.2f}")
    print("\n--- 위험 점수를 낮추기 위해 변경된 피처 값 ---")

    diff = counterfactual - instance_to_explain
    changed_indices = np.where(np.abs(diff[0]) > 1e-2)[0]

    for idx in changed_indices:
        print(f"- {X_test.columns[idx]}: {instance_to_explain[0, idx]:.2f}  ->  {counterfactual[0, idx]:.2f}")

원본 데이터의 예측 위험 점수: 31.08


IndexError: tuple index out of range